<div style="text-align: right">Name : Akshata Sanjay Jadhav</div>
<div style="text-align: right">Python version : 3.8</div>
<div style="text-align: right">Libraries used : pillow, opencv-python, openpyxl, pandas, fpdf2</div>

Following are the constants for the program. It acts like a config file in which we can change and modify values as per the requirements.

In [10]:
import pandas as pd

IMAGE_MAX_HEIGHT = 300
IMAGE_MAX_WIDTH = 1200

BORDER_WIDTH = 5
BORDER_HEIGHT = 5

PAGE_BORDER_WIDTH = 25
PAGE_BORDER_HEIGHT = 30

IMAGE_FOLDER = 'comic_images/'

NO_OF_IMAGE = 66
FONT_TEXT_SIZE = 18

DIALOGUES = pd.read_excel('comic_dialogues.xlsx',header=0)
FONTS = {'en':'fonts/BackIssuesBB_reg.ttf', 'hi':'fonts/Kalam-Bold.ttf'}


The function add_border is adding a border to the images, resize_with_borders is resizing the image to make it fit into the border. The functions combine_horizontally and combine_vertically are combining the images into horizontal strip and vertical strip to stitch the comic together.

In [11]:
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf, ImageOps as piliops

def add_border(image, color='white', width=100, height=100):
    img = piliops.expand(image.convert('RGBA'),border=(width, height),fill=color)
    return img

def resize_with_borders(images):
    total_width = sum([i.width for i in images])
    image_count = len(images)
    for i in range(image_count):
        image = images[i]
        iWidth = image.width
        width = round((IMAGE_MAX_WIDTH-BORDER_HEIGHT*2*image_count)*iWidth/total_width)
        image = image.resize((width, IMAGE_MAX_HEIGHT-BORDER_HEIGHT*2))
        image = add_border(image, 'white', BORDER_WIDTH, BORDER_HEIGHT)
        images[i] = image
        
    max_height = max([i.height for i in images])
    images = [i.resize((i.width,max_height)) for i in images]
    return images

def combine_horizontally(images):
    imgs = images
    imgs_comb = np.hstack( (np.asarray(i) for i in imgs ) )
    imgs_comb = pili.fromarray(imgs_comb)
    return imgs_comb

def combine_vertically(images):
    imgs = images
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )
    imgs_comb = pili.fromarray(imgs_comb)
    return imgs_comb
    

The function cartoonizebl_mem blurs and cartoonizes the images to make it look more like a comic.

In [12]:
import cv2 

def cartoonizebl_mem(path_in, blur, line):
    imgc = cv2.imread(path_in, cv2.IMREAD_UNCHANGED)
    line_size = line
    blur_value = blur
    gray = cv2.cvtColor(imgc, cv2.COLOR_BGR2GRAY)
    gray_blur = cv2.medianBlur(gray, blur_value)
    bigedges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, line_size, blur_value)
    bigedges_pil = cv2.cvtColor(bigedges, cv2.COLOR_BGR2RGB) # Converting BGR to RGB
    cblimg = cv2.bitwise_and(imgc, imgc, mask=bigedges)
    return pili.fromarray(cv2.cvtColor(cblimg, cv2.COLOR_BGR2RGB))

The function add_dialogue adds dialogues to the images i.e by reading the specific cell from the excel file as per language and index.

In [13]:
import textwrap

def add_dialogue(image, index, language='en'):
    caption =DIALOGUES[language][index-1]

    TINT_COLOR = (0, 0, 0)
    TRANSPARENCY = .25
    OPACITY = int(255 * TRANSPARENCY)

    img = image.convert('RGBA')

    overlay = pili.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    font = pilf.truetype(FONTS[language], FONT_TEXT_SIZE)
    text = caption
    w, h = font.getsize(text)
    width, height = img.size
    avg_char_width = sum(font.getsize(char)[0] for char in text) / len(text)
    max_char_count = int( (width-2*BORDER_WIDTH) / avg_char_width ) - 1

    dialogue_lines = []
    if language == 'en':
        h+=3
    else:
        max_char_count+=10
    for i in text.split("\n"):
        lines = textwrap.wrap(i, width=max_char_count)
        dialogue_lines.extend(lines)
        
    num_lines = len(dialogue_lines)
    text = '\n'.join(dialogue_lines)
    x, y = BORDER_WIDTH, img.height - BORDER_HEIGHT - (num_lines)*h
    #print(index,' : ',num_lines,":",dialogue_lines)
    draw.rectangle((x, y, x + img.width - BORDER_WIDTH*2, y + (num_lines)*h), fill=TINT_COLOR+(OPACITY,))
    draw.text((x, y), text, fill=(209, 239, 8), font=font)

    # Alpha composite these two images together to obtain the desired result.
    img = pili.alpha_composite(img, overlay)
    img = img.convert("RGB")
    return img


The function make_comic_page_with_language is responsible for creating random number of images in a horizontal strip and at the end combines all horizontal strips vertically to combine one single comic page.

In [14]:
import numpy as np

def make_comic_page_with_language(no_of_images, language='en', location=IMAGE_FOLDER):
    pages=[]

    list_im = [x for x in range(1,no_of_images+1)]
    while 0 < len(list_im):
        lines = 0
        strips = []
        num_cols = old_num_cols = 0
        while lines < 6 and 0 < len(list_im):
            lines += 1
            while num_cols == old_num_cols:
                num_cols = np.random.randint(3, 5)
            old_num_cols = num_cols
            window = list_im[:num_cols]

            images = [ cartoonizebl_mem(location+str(i)+'.PNG', 5, 13) for i in window]

            strip = resize_with_borders(images)
            
            strip_with_dialogues = [ add_dialogue(img, index, language) for index, img in zip(window, strip)]
            
            strips.append(combine_horizontally(strip_with_dialogues))

            list_im = list_im[num_cols:] 
        page = combine_vertically(strips)
        page = add_border(page, PAGE_BORDER_WIDTH, PAGE_BORDER_HEIGHT)
        pages.append(page)
    return pages


The function make_comic_pdf creates a pdf from the comic page achieved from the above function. 

In [15]:
from PIL import Image
from fpdf import FPDF
import os 

def make_comic_pdf(pages, language='en', cover_path='cover.png'):
    
    width, height = pages[0].size
    if os.path.exists(cover_path):
        cover = Image.open(cover_path).resize((width-PAGE_BORDER_WIDTH*2, 1800))
        cover = add_border(cover,PAGE_BORDER_WIDTH, PAGE_BORDER_HEIGHT)
        pages = [cover]+pages
        
    pdf = FPDF(unit = "pt", format = [width, height])
    
    imagelist = pages
    
    for image in imagelist:
        pdf.add_page()
        iheight = image.height        
        pdf.image(image, 0, 0, width, iheight)
    
    pdf.output(language+"_comic.pdf", "F")
    print("Published comic:"+language+"_comic.pdf")


Following are the function calls to the required functions to create a comic successfully!

In [16]:
hi_comic_pages = make_comic_page_with_language(NO_OF_IMAGE, language='hi')
make_comic_pdf(hi_comic_pages, cover_path='hi_cover.jpeg', language='hi')
en_comic_pages = make_comic_page_with_language(NO_OF_IMAGE)
make_comic_pdf(en_comic_pages, cover_path='en_cover.jpeg')


/var/folders/6_/rw1kgql93nb2dswz2brhzmxr0000gn/T/ipykernel_27443/547803987.py:24: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.hstack( (np.asarray(i) for i in imgs ) )
/var/folders/6_/rw1kgql93nb2dswz2brhzmxr0000gn/T/ipykernel_27443/547803987.py:31: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )


Published comic:hi_comic.pdf
Published comic:en_comic.pdf
